In [33]:
import pandas as pd
import numpy as np
import datetime

In [2]:
historical_data = pd.read_csv("csv_import/historical-data-020419.csv")
room_crosswalk= pd.read_csv("csv_import/listing_name_crosswalk.csv")

In [3]:
print(historical_data.head())

         Date         Type Confirmation Code  Start Date  Nights  \
0  02/04/2019       Payout               NaN         NaN     NaN   
1  02/04/2019  Reservation        HM8K2FNHDC  02/03/2019     9.0   
2  02/04/2019  Reservation        HM252EWF25  02/03/2019     4.0   
3  02/04/2019  Reservation        HMWPCM28WM  02/03/2019     5.0   
4  02/03/2019  Reservation        HM5MQNY4EW  02/02/2019     4.0   

                Guest                                           Listing  \
0                 NaN                                               NaN   
1         Amanda Long     Red Victorian - Private room w bath, Sunshine   
2         Duni Naciri     Red Victorian - Private suite w/bath, Redwood   
3  Frantisek Simancik          Red Victorian - Private room, Peace Room   
4     Israel Michetti  Red Victorian - Private room w/sink, Golden Gate   

                 Details  Reference Currency  Amount  Paid Out  Host Fee  \
0  Transfer to *****4871        NaN      USD     NaN    539.16  

In [72]:
# Add in room names
historical_data=historical_data.merge(room_crosswalk, how='left', on='Listing')

# Month and Year columns added for accounting purposes
historical_data['Month'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').month)
historical_data['Year'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').year)
data = historical_data[historical_data.Year == 2019][historical_data.Month == 1]

print(data.head())
print(data.dtypes)
print(data.Type.unique())

          Date         Type Confirmation Code  Start Date  Nights  \
18  01/15/2019  Reservation        HMHBFTP95R  01/14/2019    22.0   
38  01/31/2019       Payout               NaN         NaN     NaN   
39  01/31/2019  Reservation        HMHDNMQHEN  01/30/2019     6.0   
40  01/31/2019  Reservation        HM4N5T54YQ  01/30/2019     7.0   
41  01/31/2019       Payout               NaN         NaN     NaN   

                  Guest                                            Listing  \
18        Jamie Wilbert  Red Victorian - Private room w loftbed, Butterfly   
38                  NaN                                                NaN   
39          Jill Brophy  Red Victorian - Private suite w bath, Flower C...   
40  Ben Christoph Maass       Red Victorian - Private room -  Rainbow Room   
41                  NaN                                                NaN   

                  Details  Reference Currency      ...       Host Fee  \
18                    NaN        NaN      U

/Users/tt/.envs/jupyter-kX504egw/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [73]:
# Total paid out by month
total = data[data.Type == 'Payout']
print(total.groupby('Month')['Paid Out'].sum())

Month
1    46617.88
Name: Paid Out, dtype: float64


In [93]:
# Gross hotel revenue by listing
payouts = data[data.Type.isin(['Reservation','Adjustment'])].groupby(['Month','Listing']).agg({"Amount":np.sum}).reset_index()
print(payouts.dtypes)
print(payouts.head())
payouts.to_csv('csv_export/gross_data_2019.csv')

Month        int64
Listing     object
Amount     float64
dtype: object
   Month                                            Listing  Amount
0      1    Red Victorian - Air Bunk in shared Peacock Room  938.28
1      1        Red Victorian - Bed  in shared Friends Room  408.57
2      1  Red Victorian - Bunk #1 in shared Hole in the ...  238.62
3      1      Red Victorian - Bunk #1 in shared Poster Room  757.18
4      1  Red Victorian - Bunk #2 in shared Hole in the ...  569.20


In [34]:
# Total unassigned resolution adjustments by month
adjustments = data[data.Type.isin(['Resolution Adjustment','Resolution Payout'])].groupby('Month').agg({"Amount": np.sum})
print(adjustments)

       Amount
Month        
1     -283.33


In [37]:
# Distribute unassigned resolution adjustments by month by rooms listed that month
rooms_per_month = payouts.groupby('Month').agg({"Listing": pd.Series.nunique})
print(rooms_per_month)

type(rooms_per_month)

monthly_distribution = pd.merge(adjustments, rooms_per_month, on='Month')
monthly_distribution['Amount Per Listing'] = (monthly_distribution.Amount / monthly_distribution.Listing).round(2)
print(monthly_distribution)

monthly_distribution = monthly_distribution.drop(['Amount', 'Listing'], axis=1)
print(monthly_distribution)

       Listing
Month         
1         32.0
       Amount  Listing  Amount Per Listing
Month                                     
1     -283.33     32.0               -8.85
       Amount Per Listing
Month                    
1                   -8.85


/Users/tt/.envs/jupyter-kX504egw/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [65]:
print(payouts)

                                                           Amount
Month Listing                                                    
1     Red Victorian - Air Bunk in shared Peacock Room      938.28
      Red Victorian - Bed  in shared Friends Room          408.57
      Red Victorian - Bunk #1 in shared Hole in the Wall   238.62
      Red Victorian - Bunk #1 in shared Poster Room        757.18
      Red Victorian - Bunk #2 in shared Hole in the Wall   569.20
      Red Victorian - Bunk #2 in shared Hula Hoop Room     442.65
      Red Victorian - Bunk #2 in shared Poster Room        813.61
      Red Victorian - Bunk #3 in shared Poster Room       1075.24
      Red Victorian - Bunk #4 in shared Poster Room        728.31
      Red Victorian - Bunk#1 in shared Hula Hoop Room     1206.68
      Red Victorian - Earth Bunk in shared Peacock Room    887.05
      Red Victorian - Fire Bunk in shared Peacock Room    1171.49
      Red Victorian - Private ensuite, Rose Garden        3505.58
      Red 

In [103]:
# Create new data frame to hold adjustments for listings
adjusted_payouts = payouts.merge(monthly_distribution, how='left', on='Month')
adjusted_payouts = adjusted_payouts.fillna(0)
adjusted_payouts['Adjusted Amount'] = adjusted_payouts.Amount + adjusted_payouts['Amount Per Listing']

adjusted_payouts = adjusted_payouts.drop('Amount Per Listing', axis=1)
print(payouts.columns.values)

# Round amounts to 2 decimal points for dollars and cents (0.00)
adjusted_payouts = adjusted_payouts.round(2)

# Sum by Room
adjusted_payouts_by_room=adjusted_payouts.merge(room_crosswalk, how="left", on="Listing")
adjusted_payouts_by_room=adjusted_payouts_by_room.groupby("room-name").agg({"Amount":np.sum}).reset_index()
adjusted_payouts_by_room=adjusted_payouts_by_room.round(2)
# Save adjusted payouts to new csv file
adjusted_payouts_by_room.to_csv('csv_export/adjusted_payouts_2019.csv')

['Month' 'Listing' 'Amount']
